<a href="https://colab.research.google.com/github/Jlok17/Data620/blob/main/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project 4:

For this project we are looking at different classification methods to determine if we can predict if an email is going to be spam or "Ham" which is just a normal email. As seen below the files/folders are going to be downloaded from those website and uploaded to google drive.

From there we are going to pull that data into this notebook where the files will be read and placed into a dataframe. Each Row will correspond to the text for each email as the Ham and Spam email text are going to be in separate dataframes. Following this we are going to clean all the whitespace and non-alphabetical characters and filter the words into a more distinguish characters.

Once the data has been cleaned it will then be split up into a 60/40 ratio of training and testing data sets to be placed and a Naive Bayes Model. I used a 60/40 split as a higher yield in the training set for some reason overloaded my ram and caused this process to reset multiple times. Finally as we can see the results with a 90% accuracy and rate of 431 emails correctly labaled as spam, 132 incorrectly as ham when there were spam(False Positive), 26 labeled incorrectly as spam when they were ham(False Negative) and 991 that were correctly labeled as Ham.


---


My data source is: [Spam/Ham Dataset](https://spamassassin.apache.org/old/publiccorpus/
)


> It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.
For this project, you can either use the above dataset to predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).For more adventurous students, you are welcome (encouraged!) to come up a different set of documents (including scraped web pages!?) that have already been classified (e.g. tagged), then analyze these documents to predict how new documents should be classified.


In [61]:
import re
import pandas as pd
from nltk.corpus import stopwords
import os
from pandas import DataFrame
from functools import partial
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
import nltk
nltk.download('stopwords', quiet=True)

True

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
# Set the path to your folder within your Google Drive
folder_path = '/content/drive/MyDrive/'

# List the contents of the folder
contents = os.listdir(folder_path)


In [40]:
# Set the path to the "spam_2" folder
folder_path = '/content/drive/MyDrive/spam_2'
# List the files in the directory
files = os.listdir(folder_path)

num_files = len(files)
print("Number of files:", num_files)

Number of files: 1397


In [41]:
# Set the path to the "easy_ham" folder
folder_path2 = '/content/drive/MyDrive/easy_ham'
# List the files in the directory
files = os.listdir(folder_path2)

num_files2 = len(files)
print("Number of files:", num_files2)

Number of files: 2551


In [43]:
# Set the path to the directory containing the spam files
spam_directory = '/content/drive/MyDrive/spam_2'

# Set the path to the directory containing the ham files
ham_directory = '/content/drive/MyDrive/easy_ham'


# Creating a function to read each file in the folder
def read_emails(file_path, class_value):
    with open(file_path, 'r', encoding='latin-1') as file:
        lines = file.readlines()
        text = ' '.join(lines)
        return {'file': file_path, 'text': text, 'class': class_value, 'spam': int(class_value == 'spam')}

# Getting Ham and Spam Files
spam_files = [os.path.join(spam_directory, file) for file in os.listdir(spam_directory)]
ham_files = [os.path.join(ham_directory, file) for file in os.listdir(ham_directory)]

# Creating Dataframes
spam_df = pd.DataFrame(map(partial(read_emails, class_value='spam'), spam_files))
ham_df = pd.DataFrame(map(partial(read_emails, class_value='ham'), ham_files))
emails = pd.DataFrame.append(spam_df, ham_df, ignore_index=True)

<ipython-input-43-297b17b409ce>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emails = DataFrame.append(spam_df, ham_df, ignore_index=True)


In [7]:
emails.head

<bound method NDFrame.head of                                                    file  \
0     /content/drive/MyDrive/spam_2/00566.9b3e6f0ed5...   
1     /content/drive/MyDrive/spam_2/00610.d78eda68ed...   
2     /content/drive/MyDrive/spam_2/00311.176626eb0e...   
3     /content/drive/MyDrive/spam_2/00314.ce1926a981...   
4     /content/drive/MyDrive/spam_2/00496.acf53035be...   
...                                                 ...   
3943  /content/drive/MyDrive/easy_ham/0152.10d322018...   
3944  /content/drive/MyDrive/easy_ham/1622.91772e8eb...   
3945  /content/drive/MyDrive/easy_ham/0155.56e909508...   
3946  /content/drive/MyDrive/easy_ham/1636.5583afc73...   
3947  /content/drive/MyDrive/easy_ham/1561.b968a0929...   

                                                   text class  spam  
0     From egtan@yahoo.com  Mon Jun 24 17:51:57 2002...  spam     1  
1     From moreinformation@btamail.net.cn  Mon Jun 2...  spam     1  
2     From StopSearching@fuse.net  Mon Jun 24 17:03

In [9]:
spamham = emails

In [44]:
# Cleaning the text column by removing line breaks and tabs from the different files
spamham['text'] = spamham['text'].str.replace(r'[\r\n\t]+', '')

# Removing punctuation from the text columns
spamham['text'] = spamham['text'].str.replace(r'[^\w\s]', ' ')

# Cleaning data by removing common words, punctuation, and numbers from the text files of the emails
corpus = spamham['text']
corpus = corpus.str.lower()
corpus = corpus.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))
corpus = corpus.apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
corpus = corpus.apply(lambda x: re.sub(r'\d+', '', x))
corpus = corpus.str.strip()

# Creating a Document-Term Matrix (DTM) from the corpus object to represent the term frequency for the documents
cv = CountVectorizer(min_df=0.1, max_df=0.9)
dtm = cv.fit_transform(corpus)

<ipython-input-44-2150d80667b1>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  spamham['text'] = spamham['text'].str.replace(r'[\r\n\t]+', '')
<ipython-input-44-2150d80667b1>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  spamham['text'] = spamham['text'].str.replace(r'[^\w\s]', ' ')


In [45]:
# Creating a Document-Term Matrix (DTM)
cv = CountVectorizer()
dtm = cv.fit_transform(corpus)
terms = cv.get_feature_names_out()


In [46]:
# Converting the DTM into a DataFrame
email_dtm = pd.DataFrame(dtm.toarray(), columns= terms )
email_dtm['class'] = spamham['class']
email_dtm['class'] = pd.Categorical(email_dtm['class'])


In [48]:
# The training set will use 60% of the total rows and a higher row count crashed my computer
sample_size = int(0.6 * email_dtm.shape[0])

# Setting Seed for reproducibility
np.random.seed(334)

indices = np.random.choice(email_dtm.shape[0], size=sample_size, replace=False)

# Categorize between the training and testing DTMs
dtm_training = email_dtm.iloc[indices, :]
dtm_testing = email_dtm.drop(indices, axis=0)

# Training and Test Spam labels
training_labels = dtm_training['class']
testing_labels = dtm_testing['class']

# Propotion between Ham and Spam Amount
training_label_proportions = training_labels.value_counts(normalize=True)
print(training_label_proportions)


ham     0.647804
spam    0.352196
Name: class, dtype: float64


In [54]:
email_dtm.columns

Index(['__', '___', '____', '_____', '______', '_______', '________',
       '_________', '__________', '___________',
       ...
       'þày', 'þë', 'þö', 'þööööà', 'ÿò', 'ÿÿ', 'ÿÿã',
       'ÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿò',
       'ÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿô',
       'ÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿó'],
      dtype='object', length=74364)

In [57]:
# Convert the Occurrence Matrix to a binary matrix
dtm_training.loc[:, 2:3816] = np.where(dtm_training.iloc[:,2:3816] == 0, "No", "Yes")
dtm_testing.loc[:, 2:3816] = np.where(dtm_testing.iloc[:, 2:3816] == 0, "No", "Yes")


<ipython-input-57-caa0e18a9692>:2: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  dtm_training.loc[:, 2:3816] = np.where(dtm_training.iloc[:,2:3816] == 0, "No", "Yes")
<ipython-input-57-caa0e18a9692>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtm_training.loc[:, 2:3816] = np.where(dtm_training.iloc[:,2:3816] == 0, "No", "Yes")
<ipython-input-57-caa0e18a9692>:3: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  dtm_testing.loc[:, 2:3816] = np.where(dtm_testing.iloc[:, 2:3816] == 0, "No", "Yes")


In [58]:
dtm_training_1 = dtm_training.replace({'Yes': 1, 'No': 0})
dtm_testing_1 = dtm_testing.replace({'Yes': 1, 'No': 0})

# Naive Bayes Predictive Model
model_classifier = MultinomialNB()
model_classifier.fit(dtm_training_1.iloc[:, 2:3817].astype(int), training_labels)

# Using model on the testing dataframe
test_prediction = model_classifier.predict(dtm_testing_1.iloc[:, 2:3817].astype(int))

# Performance of the prediction model
cm = confusion_matrix(testing_labels, test_prediction, labels=["spam", "ham"])
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[431 132]
 [ 26 991]]


In [59]:
accuracy = accuracy_score(testing_labels, test_prediction)
print("Accuracy:", accuracy)

Accuracy: 0.9
